In [56]:
import pandas as pd

In [57]:
# зібрати всі дані в df
df_bus_stantion = pd.read_csv("./data/Bus Stantion + Park_ Майбутнє туризму у Володимирі_ думка мешканців та гостей міста .csv") #bus stanion
df_central_square = pd.read_csv("./data/Central square_ Майбутнє туризму у Володимирі_ думка мешканців та гостей міста .csv") #central square
df_linkedin = pd.read_csv("./data/Lkdn_ Майбутнє туризму у Володимирі_ думка мешканців та гостей міста .csv") #linkedin
df_social_media = pd.read_csv("./data/Володимир Медіа_ Майбутнє туризму у Володимирі_ думка мешканців та гостей міста .csv") #internet
df_kogorta = pd.read_csv("./data/Когорта_ Майбутнє туризму у Володимирі_ думка мешканців та гостей міста .csv") #individual groups

In [58]:
def add_origin_column(df, column_name):
    df["origin"] = column_name
    df = df.rename(
        columns={
            "Стать респондента": "sex", 
            "Рік народження": "age",
            "Скільки часу ви проживаєте у місті Володимир": "нears_lived",
            "Ваш рід занять": "occupation",
            "Рівень доходу": "sallary",
            "Яку частину доходу ви витрачаєте на діяльсть пов'язану з туризмом та сферою обслуговування ": "Income_spent_on_tourism",
            "Як ви ставитесь до візитів хасидів до Володимира?": "hasidic_loyalty",
            "Як ви ставитесь до туристів з Польщі?": "polish_loyalty",
            "Як ви ставитесь до туристів з інших країн?":"other_loyalty",
            "На вашу думку який у вас рівень володіння англійською мовою?":"english_level",
            "На вашу думку який у вас рівень володіння польською мовою?":"polish_level",
            "Чи достатньо, на вашу думку, у Володимирі готелей, для розміщення туристів чи туристичних груп?":"holels_sits",
            "Чи достатньо, на вашу думку, у Володимирі закладів харчування?":"caffee_sits",
            "Чи вважаєте ви, що Володимир достатньо представлений в інтернеті як туристичне місто?":"internet_representation",
            " Які туристичні об’єкти у Володимирі є, на вашу думку, ключовими для приваблення туристів?":"key_items",
            "Які зміни в туристичній інфраструктурі ви вважаєте найбільш необхідними? (можна кілька варіантів)":"key_changes",
            "Які об’єкти, на вашу думку, слід відкрити чи розвинути у Володимирі, щоб туристи залишалися довше? (можна кілька варіантів)":"tourism_development",
            
            })
    return df

In [59]:
df_bus_stantion = add_origin_column(df_bus_stantion, "bus stantion")
df_central_square = add_origin_column(df_central_square, "central_square")
df_linkedin = add_origin_column(df_linkedin, "linkedin")
df_social_media = add_origin_column(df_social_media, "social media")
df_kogorta = add_origin_column(df_kogorta, "test group")

# df_social_media.head()
df_kogorta.columns


Index(['Timestamp', 'sex', 'age', 'нears_lived', 'occupation', 'sallary',
       'Income_spent_on_tourism', 'hasidic_loyalty', 'polish_loyalty',
       'other_loyalty', 'english_level', 'polish_level', 'holels_sits',
       'caffee_sits', 'internet_representation', 'key_items', 'key_changes',
       'tourism_development', 'origin'],
      dtype='object')

In [60]:
df_kogorta.head()

,Timestamp,sex,age,нears_lived,occupation,sallary,Income_spent_on_tourism,hasidic_loyalty,polish_loyalty,other_loyalty,english_level,polish_level,holels_sits,caffee_sits,internet_representation,key_items,key_changes,tourism_development,origin
0,2025/08/29 11:45:32 PM GMT+3,Жіноча,1987.0,більше 20 років,NaN,10 000 грн - 20 000 грн,5%-10%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ні,Вали городища,Покращення доріг і транспортного сполучення;Бі...,Рекреаційна зона відпочинку з водоймою;Музей з...,test group
1,2025/08/29 11:47:27 PM GMT+3,Жіноча,NaN,NaN,Приватний підприємець,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Покращення доріг і транспортного сполучення;Бі...,Кав’ярні/фуд-корт із місцевою кухнею;Контактни...,test group
2,2025/08/29 11:49:01 PM GMT+3,Жіноча,1973.0,більше 20 років,Працівник,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Покращення доріг і транспортного сполучення;Ор...,Кав’ярні/фуд-корт із місцевою кухнею;Контактни...,test group
3,2025/08/29 11:50:46 PM GMT+3,Жіноча,NaN,більше 20 років,Працівник,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Краще туристичне зонування та навігація (вказі...,Кав’ярні/фуд-корт із місцевою кухнею;Музей з і...,test group
4,2025/08/29 11:52:42 PM GMT+3,Жіноча,1964.0,більше 20 років,Пенсіонер,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Частково,NaN,Покращення доріг і транспортного сполучення;Бі...,Контактний зоопарк / міні-ферма для дітей;SPA-...,test group
